<a href="https://colab.research.google.com/github/AhirtonLopes/TDCBH_Demos/blob/master/Demo1_GeneralRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/python
# coding: utf-8

#General Purpose Recommendation - Python - Jose Ahirton Lopes

"""
__author__ = "Ahirton Lopes"
__copyright__ = "Copyright 2018"
__credits__ = ["Ahirton Lopes"]
__license__ = "None"
__version__ = "1.0"
__maintainer__ = "Ahirton Lopes"
__email__ = "ahirtonlopes@gmail.com"
__status__ = "Beta"
"""

#Imports diversos, numpy para operações com matrizes e vetores 

import pandas as pd
import numpy as np
import unicodedata
import pandas as pd

#Opcoes para print de todos os dados do dataset

pd.set_option("display.max_columns", 15)
pd.set_option("display.width", 200)

#Criacao de dataframe via dataset

df = pd.read_csv('Desktop/ecommerce_data.csv', encoding="ISO-8859-1", dtype={'CustomerID': str, 'InvoiceID': str})

#print df

In [0]:
#Criacao de dataframe de vendas agrupando itens (com respectivas descricao e preco total)

dfsales = df.groupby('Description')['UnitPrice'].sum()
dfsales = pd.DataFrame(dfsales)
dfsales = dfsales.sort_values('UnitPrice', ascending=False)
dfsales['Description'] = dfsales.index

dfsales = dfsales[0:100]

#print dfsales

columns_to_keep = ['Description', 'CustomerID']
df1 = df[columns_to_keep]
uniqueproducts = dfsales['Description'].unique()

halfproducts = uniqueproducts[4:100]  
df1 = df1.where(df1['Description'].isin(halfproducts)).dropna()
df1 = df1.set_index('CustomerID')

#print df1

df2 = pd.get_dummies(df1)
df2 = df2.reset_index()

#print df2

uniqueproducts = df2.columns.values

#print uniqueproducts

listoflists = []

for item in uniqueproducts[1:]:
    df3 = df2.groupby('CustomerID')[item].max()
    listoflists.append(df3.values)
    customerlist = df3.index

newdf = pd.DataFrame(listoflists)
newdf['Product Name'] = uniqueproducts[1:]
newdf = newdf.set_index('Product Name')
newdf = newdf.T
newdf['CustomerID'] = customerlist
newdf = newdf.set_index('CustomerID')

#print newdf

In [0]:
#Etapa de Teste - Loop iterando por cada coluna agrupando os scores de teste para cada produto
#Recall é utilizado como medida para score de teste referente ao modelo

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier  

listofproducts = []
listofrecall = []

for column, name in enumerate(newdf.columns.values):
    columntotest = column
    excludedcolumn = newdf.columns.values[columntotest]
    X_sales = newdf.iloc[:, 0:]
    X_sales = X_sales.drop(excludedcolumn, axis=1, level=None, inplace=False, errors='raise')
    y_sales = newdf.iloc[:, columntotest]
    X_train, X_test, y_train, y_test = train_test_split(X_sales, y_sales, random_state=0)
    
    #Utilizando arvore de decisao para calculo de scores de recomendação de cada produto p/ cada cliente 
    clf = DecisionTreeClassifier(max_depth=10, random_state=0).fit(X_train, y_train)
    clf_predicted = clf.predict(X_test)
    confusion = confusion_matrix(y_test, clf_predicted)
    precision = precision_score(y_test, clf_predicted)
    recall = recall_score(y_test, clf_predicted)
    #print(excludedcolumn)
    #print('Precision Score: {:.2f}'.format(precision))
    #print('Recall Score: {:.2f}'.format(recall))
    listofproducts.append(excludedcolumn)
    listofrecall.append(recall)
accuracydf = pd.DataFrame({'Recommendation': listofproducts, 'Score': listofrecall})

accuracydf.mean()

/Users/ahirtonlopes/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/ahirtonlopes/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Score    0.281022
dtype: float64

In [0]:
#Calculo de probabilidades - Loop iterando todos os clientes e produtos.

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

mastervaluelist = []
masterproductlist = []
masterprobabilitylist = []

columnvalues = newdf.columns.values
columnvalues = columnvalues[0:-1]

In [0]:
for column, name in enumerate(columnvalues):
    #print('predicting product number: ', column, ':', name)

    columntotest = column
    excludedcolumn = newdf.columns.values[columntotest]
    X_sales = newdf.iloc[:, 0:]
    X_sales = X_sales.drop(excludedcolumn, axis=1, level=None, inplace=False, errors='raise')
    y_sales = newdf.iloc[:, columntotest]
    X_train, X_test, y_train, y_test = train_test_split(X_sales, y_sales, random_state=0)
    clf = DecisionTreeClassifier(max_depth=10, random_state=0).fit(X_train, y_train)
    lookforcustomers = X_test.where(newdf[excludedcolumn] == 0).dropna()

    clf_predicted = clf.predict(lookforcustomers)
    logs = clf.predict_proba(lookforcustomers)

    probabilitylist = []
    for item in logs:
        try:
            probabilitylist.append(float(item[1]))
        except:
            probabilitylist.append(float(0.0))

    predicteddf = pd.DataFrame({'Customer Tested': lookforcustomers.index, 'Prediction': probabilitylist})
    predicteddf = predicteddf.set_index('Customer Tested')
    predictedvalues = predicteddf.index.values
    predictedvalues = predictedvalues.tolist()
    mastervaluelist = mastervaluelist + predictedvalues
    predictedproduct = [excludedcolumn] * len(predictedvalues)
    masterproductlist = masterproductlist + predictedproduct
    masterprobabilitylist = masterprobabilitylist + probabilitylist

#Organizando todos os clientes, probabilidades, produtos e scores em um so dataframe para consulta via front.

finaldf = pd.DataFrame(
    {'Customer': mastervaluelist, 'Recommendation': masterproductlist, 'Probability': masterprobabilitylist})

finalfinaldf = pd.merge(finaldf, accuracydf, on='Recommendation', how='outer')
finalfinaldf.sort_values(['Customer', 'Score'], ascending=False)

finalfinaldf['WeightedScore'] = finalfinaldf['Probability'] * finalfinaldf['Score']

finalfinaldf = finalfinaldf.where(finalfinaldf['WeightedScore'] != 0).dropna().sort_values(
    ['Customer', 'WeightedScore'], ascending=False)
finalfinaldf.groupby('Customer').head(3)

print(finalfinaldf)

      Customer  Probability                                   Recommendation     Score  WeightedScore
3265     18280     0.567568           Description_ALARM CLOCK BAKELIKE GREEN  0.722892       0.410290
4133     18280     0.241379            Description_ALARM CLOCK BAKELIKE PINK  0.671642       0.162120
58316    18280     0.079900             Description_REGENCY CAKESTAND 3 TIER  0.399038       0.031883
77391    18280     0.050000   Description_WOOD 2 DRAWER CABINET WHITE FINISH  0.490566       0.024528
75678    18280     0.093773   Description_WHITE HANGING HEART T-LIGHT HOLDER  0.239437       0.022453
50477    18280     0.085171                        Description_PARTY BUNTING  0.233533       0.019890
49691    18280     0.083597      Description_PAPER CHAIN KIT 50'S CHRISTMAS   0.222222       0.018577
41089    18280     0.040662              Description_JUMBO BAG RED RETROSPOT  0.433962       0.017646
6698     18280     0.089256        Description_BAKING SET 9 PIECE RETROSPOT   0.17

In [0]:
finalfinaldf.to_csv("Desktop/recomendacaofinal.csv",index  = False)